In [79]:
import numpy as np
import matplotlib.pyplot as plt
import random

In [72]:
def pattern_maker(N, alpha):
    p = int(alpha*N)

    x = np.zeros((p, N))
    for mu in range(p):
        for i in range(N):
            random_var = np.random.uniform()
            if random_var < 0.5:
                x[mu, i] += -1
            else:
                x[mu, i] += 1
    
    return x

def connection_weights(x):
    p, N = x.shape
    w = np.dot(x.T, x) / N
    np.fill_diagonal(w, 0)
    return w

def update_parallel(w, S):
    return np.sign(np.dot(w, S))

def update_sequential(w, S):
    # Recorro los índices de forma aleatoria
    indices = list(range(len(S))) 
    random_indices = random.sample(indices, len(indices))
    S_new = np.copy(S)
    for i in random_indices:
        S_new[i] = np.sign(np.dot(w[:, i], S_new))
    return S_new


def dynamics(N, alpha, update=update_sequential, iterations = 1000):
    x = pattern_maker(N, alpha)
    w = connection_weights(x)
    p = int(N*alpha)

    m = np.zeros(p)
    time = np.zeros(p)

    for mu in range(p):
        t = 0
        S = np.copy(x[mu])

        for _ in range(iterations):
            t = 1
            S_new = update(w, S)
            if np.array_equal(S_new, S):
                break
            S = np.copy(S_new)
    
        time[mu] = t
        m[mu] = (1/N) * np.dot(S.T, x[mu])
    
    return m, time

In [ ]:
# Secuencial
N = [500, 1000, 2000, 4000]
alpha = [0.12, 0.14, 0.16, 0.18]

m = np.zeros((len(N), len(alpha)), dtype=object)
t = np.zeros((len(N), len(alpha)))

for i in range(len(N)):
    for j in range(len(alpha)):
        m[i,j], t[i,j] = dynamics(N[i], alpha[j])
        print('Finalizado caso ', [N[i], alpha[j]])

In [ ]:
figs, axs = plt.subplots(len(N), len(alpha), figsize=(12, 12), sharex=True)

for i in range(len(N)):
    for j in range(len(alpha)):
        axs[i,j].hist(m[i,j], bins=40, range=(0,1), edgecolor='black')

        if i == 0:
            axs[i,j].set_title(f'$\\alpha = {alpha[j]}$')
        if j == 0:
            axs[i,j].set_ylabel(f'$N = {N[i]}$')
        if i == len(N) - 1:
            axs[i,j].set_xlabel(f'$m$')
plt.tight_layout()

In [ ]:
# Paralelo
N = [500, 1000, 2000, 4000]
alpha = [0.12, 0.14, 0.16, 0.18]

m_p = np.zeros((len(N), len(alpha)), dtype=object)
t_p = np.zeros((len(N), len(alpha)))


for i in range(len(N)):
    for j in range(len(alpha)):
        m[i,j], t_p[i,j] = dynamics(N[i], alpha[j], update=update_parallel)
        print('Finalizado caso ', [N[i], alpha[j]])

In [ ]:
figs, axs = plt.subplots(len(N), len(alpha), figsize=(12, 12), sharex=True)

for i in range(len(N)):
    for j in range(len(alpha)):
        axs[i,j].hist(m_p[i,j], bins=40, range=(0,1), edgecolor='black')

        if i == 0:
            axs[i,j].set_title(f'$\\alpha = {alpha[j]}$')
        if j == 0:
            axs[i,j].set_ylabel(f'$N = {N[i]}$')
        if i == len(N) - 1:
            axs[i,j].set_xlabel(f'$m$')
plt.tight_layout()

In [ ]:
figs, axs = plt.subplots(1, 2, figsize=(10,5), sharex=True)

for i in range(len(N)):
    axs[0].plot(alpha, t[i], 'o-', label=f'N = {N[i]}')
    axs[1].plot(alpha, t_p[i], 'o-', label=f'N = {N[i]}')

axs[0].set_xlabel(f'$\\alpha$')
axs[0].set_ylabel('Iteración')
axs[0].set_ylim(0, 1000)
axs[0].legend()
axs[1].set_xlabel(f'$\\alpha$')
axs[1].set_ylabel('Iteración')
axs[1].set_ylim(0, 1000)
axs[1].legend()
plt.tight_layout()